สมุดบันทึกต่อไปนี้ถูกสร้างขึ้นโดยอัตโนมัติโดย GitHub Copilot Chat และมีไว้สำหรับการตั้งค่าเบื้องต้นเท่านั้น


# แนะนำเกี่ยวกับการออกแบบพรอมต์

การออกแบบพรอมต์คือกระบวนการสร้างและปรับแต่งพรอมต์สำหรับงานประมวลผลภาษาธรรมชาติ โดยจะต้องเลือกพรอมต์ที่เหมาะสม ปรับค่าต่าง ๆ และประเมินประสิทธิภาพของพรอมต์นั้น ๆ การออกแบบพรอมต์มีความสำคัญอย่างมากในการช่วยให้โมเดล NLP มีความแม่นยำและทำงานได้อย่างมีประสิทธิภาพ ในหัวข้อนี้ เราจะมาทำความเข้าใจพื้นฐานของการออกแบบพรอมต์โดยใช้โมเดลของ OpenAI เป็นตัวอย่างในการศึกษา


### แบบฝึกหัดที่ 1: การแบ่งคำ (Tokenization)
สำรวจการแบ่งคำโดยใช้ tiktoken ซึ่งเป็นตัวแบ่งคำที่รวดเร็วและเป็นโอเพ่นซอร์สจาก OpenAI  
ดูตัวอย่างเพิ่มเติมได้ที่ [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst)


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### แบบฝึกหัดที่ 2: ตรวจสอบการตั้งค่า OpenAI API Key

รันโค้ดด้านล่างเพื่อตรวจสอบว่า endpoint ของ OpenAI ของคุณตั้งค่าเรียบร้อยแล้วหรือไม่ โค้ดนี้จะทดสอบด้วย prompt ง่าย ๆ และตรวจสอบผลลัพธ์ที่ได้ โดยอินพุต `oh say can you see` ควรจะต่อเนื่องไปในแนวทางประมาณ `by the dawn's early light..`


In [ ]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

## Updated
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt)
print(response)

### แบบฝึกหัดที่ 3: การแต่งเรื่องขึ้นมา

ลองสำรวจดูว่าเกิดอะไรขึ้นเมื่อคุณขอให้ LLM ตอบกลับเกี่ยวกับหัวข้อที่อาจไม่มีอยู่จริง หรือหัวข้อที่มันอาจไม่รู้จักเพราะอยู่นอกชุดข้อมูลที่ถูกฝึกมา (ข้อมูลใหม่กว่า) สังเกตดูว่าคำตอบจะเปลี่ยนไปอย่างไรถ้าคุณลองใช้ข้อความแจ้งที่ต่างออกไป หรือเปลี่ยนไปใช้โมเดลอื่น


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt)
print(response)

### แบบฝึกหัดที่ 4: ตามคำสั่ง
ใช้ตัวแปร "text" เพื่อกำหนดเนื้อหาหลัก
และใช้ตัวแปร "prompt" เพื่อให้คำแนะนำที่เกี่ยวข้องกับเนื้อหาหลักนั้น

ในที่นี้ เราขอให้โมเดลสรุปเนื้อหาให้กับนักเรียนชั้นประถมศึกษาปีที่ 2


In [ ]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt)
print(response)

### แบบฝึกหัดที่ 5: คำขอที่ซับซ้อน
ลองส่งคำขอที่มีข้อความจากระบบ, ผู้ใช้ และผู้ช่วย
ระบบจะตั้งค่าบริบทให้กับผู้ช่วย
ข้อความจากผู้ใช้และผู้ช่วยจะสร้างบริบทการสนทนาแบบหลายรอบ

สังเกตว่าบุคลิกของผู้ช่วยถูกตั้งค่าให้ "ประชดประชัน" ในบริบทของระบบ
ลองใช้บริบทบุคลิกแบบอื่น หรือเปลี่ยนชุดข้อความระหว่างผู้ใช้และผู้ช่วยดู


In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

### แบบฝึกหัด: สำรวจสัญชาตญาณของคุณ
ตัวอย่างข้างบนนี้ให้รูปแบบที่คุณสามารถนำไปใช้สร้าง prompt ใหม่ ๆ ได้ (ทั้งแบบง่าย แบบซับซ้อน แบบคำสั่ง ฯลฯ) - ลองสร้างแบบฝึกหัดอื่น ๆ เพื่อสำรวจแนวคิดอื่น ๆ ที่เราได้พูดถึง เช่น ตัวอย่าง คำใบ้ และอื่น ๆ



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารฉบับนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามอย่างเต็มที่เพื่อความถูกต้อง แต่โปรดทราบว่าการแปลโดยระบบอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่มีความสำคัญ แนะนำให้ใช้บริการแปลโดยนักแปลมืออาชีพ ทางเราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่คลาดเคลื่อนซึ่งเกิดจากการใช้การแปลนี้
